In [111]:
surface = 'utrecht'
# wiki = 'simplewiki-20211120'
wiki = 'nlwiki-20220301'
modelfile = f'wiki/{wiki}/experiments/clean-q0.25.32b.vw'
datafile = f'wiki/{wiki}/experiments/clean-q0.25.dat'

import subprocess
args = ["minimel", "audit", modelfile, datafile, surface ]
feats = set()
with subprocess.Popen(args, stdout=subprocess.PIPE, stderr=None) as process:
    for line in process.stdout:
        line = line.decode('utf8').rstrip()
        if line.startswith('\t'):
            feats.update(set(f for f in line[1:].split('\t') if f.startswith('l^')))
len(feats)

creating quadratic features for pairs: ls
only testing
using no cache
Reading datafile = none
num sources = 0
Num weight bits = 32
learning rate = 0.5
initial_t = 0
power_t = 0.5
Enabled reductions: gd, scorer-identity, csoaa_ldf-prob, shared_feature_merger
Input label = cs
Output pred = prob
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
Traceback (most recent call last):
  File "/home/bbkruit/scratch/anaconda3/envs/takco2/bin/minimel", line 33, in <module>
    sys.exit(load_entry_point('minimel', 'console_scripts', 'minimel')())
  File "/var/scratch/bbkruit/minimal-EL/minimel/__main__.py", line 70, in main
    defopt.run(subcommands)
  File "/home/bbkruit/scratch/anaconda3/envs/takco2/lib/python3.9/site-packages/defopt.py", line 275, in run
    return func(*ba.args, **ba.kwargs)
  File "/var/scratch/bbkruit/minimal-EL/minimel/audit.py", line 13, in audit
    mode

253045

In [112]:
import io
import pandas as pd

df = pd.read_csv(
    io.StringIO('\n'.join(feats)), 
    sep="\*|:|\^|=", 
    header=None,
    engine="python",
    usecols=[2,4,7],
    names=['wid', 'feat', 'weight']
).dropna()

In [114]:
select_ents = set(df['wid'].unique())

import sqlite3

con = sqlite3.connect(f'wiki/{wiki}/index_{wiki}.db')
ent_label = {}
for e in select_ents:
    l = pd.read_sql_query(f'select * from mapping where wikidata_id="Q{e}" limit 1', con)
    ent_label[e] = l['wikipedia_title'][0]
print(ent_label)

{18108: 'Utrecht_(Zuid-Afrika)', 803: 'Utrecht_(stad)', 2679365: 'Heerlijkheid_Utrecht', 575655: 'Station_Utrecht_Centraal', 24680: 'FC_Utrecht', 776: 'Utrecht_(provincie)', 2012748: 'Vechtsebanen', 261716: 'Aartsbisdom_Utrecht_(rooms-katholiek)', 221653: 'Universiteit_Utrecht', 707767: 'Sticht_Utrecht', 847384: 'Utrechts_Conservatorium', 2193594: 'Hr.Ms._Utrecht_(1901)', 85308316: 'BVC_Utrecht'}


In [115]:
def topfeat(gr):
    gr = gr.drop(columns='wid').set_index('feat').dropna()
    gr = gr.loc[gr['weight'].apply('abs').sort_values().index[::-1]]
    return gr.head(10).reset_index()

tops = df.groupby('wid').apply(topfeat)
tops = tops.swaplevel().unstack().swaplevel(axis=1).sort_index(axis=1).T
tops.index = tops.index.set_levels([ent_label[wid] for wid in tops.index.levels[0]], level=0)
tops

0            1  \
wid                                                                       
Utrecht_(provincie)                   feat       provincie         stad   
                                      weight      -1.54169     0.803052   
Utrecht_(stad)                        feat       provincie         stad   
                                      weight       1.50596    -0.678369   
Utrecht_(Zuid-Afrika)                 feat       categorie   nederlands   
                                      weight       1.26529     0.853828   
FC_Utrecht                            feat       categorie      utrecht   
                                      weight       1.16863     0.893371   
Universiteit_Utrecht                  feat       provincie     gemeente   
                                      weight       1.16036     0.766727   
Aartsbisdom_Utrecht_(rooms-katholiek) feat         utrecht    geografie   
                                      weight      0.848614     0.711323   
Station_Utrecht_Centraal              feat      nederlands          hij   
                                      weight        1.2553     0.799994   
Sticht_Utrecht                        feat    gemeentelijk  nederlandse   
                                      weight      0.786053     0.785903   
Utrechts_Conservatorium               feat         utrecht    categorie   
                                      weight      0.972213     0.969662   
Vechtsebanen                          feat         utrecht    categorie   
                                      weight       1.18081     0.961221   
Hr.Ms._Utrecht_(1901)                 feat         utrecht    categorie   
                                      weight       1.03316      1.02736   
Heerlijkheid_Utrecht                  feat    gemeentelijk      utrecht   
                                      weight      0.862723     0.815876   
BVC_Utrecht                           feat              is      utrecht   
                                      weight      0.887466     0.828952   

                                                         2              3  \
wid                                                                         
Utrecht_(provincie)                   feat           baarn      geografie   
                                      weight     -0.736259      -0.699101   
Utrecht_(stad)                        feat         utrecht          baarn   
                                      weight     -0.640243       0.627676   
Utrecht_(Zuid-Afrika)                 feat              is           brug   
                                      weight      0.788975       0.673091   
FC_Utrecht                            feat            stad   wakkerendijk   
                                      weight      0.696396       0.634286   
Universiteit_Utrecht                  feat        bisschop  rijksmonument   
                                      weight      0.761021       0.750109   
Aartsbisdom_Utrecht_(rooms-katholiek) feat      waterschap   gemeentelijk   
                                      weight      0.691718       0.650602   
Station_Utrecht_Centraal              feat       geografie     waterschap   
                                      weight      0.778287       0.701667   
Sticht_Utrecht                        feat      buurtschap         museum   
                                      weight      0.780547       0.739906   
Utrechts_Conservatorium               feat    wakkerendijk            een   
                                      weight      0.662973       0.580366   
Vechtsebanen                          feat              in   wakkerendijk   
                                      weight      0.745774       0.659443   
Hr.Ms._Utrecht_(1901)                 feat              is   gemeentelijk   
                                      weight      0.706559       0.659831   
Heerlijkheid_Utrecht                  feat              is  rijksmonument   
                               